In [51]:
import os

In [52]:
%pwd

'd:\\'

In [53]:
os.chdir('../')

In [54]:
%pwd

'd:\\'

In [55]:
# entity

from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_url: str 
    local_data_file: Path 
    unzip_dir: Path

In [56]:
# constants
from pathlib import Path 
config_file_path = Path("config/config.yaml")
params_file_path = Path("params.yaml")
schema_file_path = Path("schema.yaml")

In [57]:
# configurationmanager
from mlProject.constants import*
from mlProject.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=config_file_path,
        params_filepath=params_file_path,
        schema_filepath=schema_file_path):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion 

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [58]:
# components

import os
import urllib.request as request 
import zipfile
from mlProject import logger 
from mlProject.utils.common import get_size

In [59]:
# componenst 2
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config 
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url=self.config.source_url,
                filename =self.config.local_data_file
            )
            logger.info(f"{filename} download with following into :\{headers}")
        else:
            logger.info(f"file alredy exist of size:{get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [60]:
#pipeline

try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config() # cm2ndpart
    data_ingestion=DataIngestion(config=data_ingestion_config) # component 
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'